In [ ]:
import datetime
import itertools

In [ ]:
import numpy as np

In [ ]:
from sairen import MarketEnv
from sairen.xform import BinaryDelta
interations = 10
episodes = 30
step_per_episode = 100

In [ ]:
class ContinuousActionLinearPolicy:
    def _init_(self, theta, n_in, n_out):
        assert len(theta) == (n_in + 1) * n_out, format(n_in, n_out, len(theta))
        self.w = theta[0:n_in * n_out].reshape(n_in, n_out)
        self.b = theta[n_in * n_out:None].reshape(1, n_out)
        
    def act(self, ob):
        return np.squeeze(np.asarray(ob).dot(self.w) + self.b)

In [ ]:
def cem(eval_func, params_mean, batch_size, n_iter, elite_frac, params_std=1.0):
    n_elite = int(np.round(batch_size * elite_frac))
    params_std = np.ones_like(params_mean) * params_std
    
    for _ in range(n_iter):
        params = np.array([params_mean + deth for dth in params_std[None, :] * np.randon.randn(batch_size, params_mean.size)])
        rewards = np.array([eval_func(p, ep) for ep, p in enumerate(params)])
        elite_inds = rewards.argsort()[:n_elite]
        elite_params = params[elite_inds]
        params_mean = elite_params.mean(axis=0)
        params_std = elite_params.std(axis=0)
        yield{'pstsmd_mean': params, 'rewards': rewards, 'params_best': elite_params[0], 'reward_best': rewards.max(), 'reward_elite': rewards[elite_inds].mean(), 'elite_mean': params_mean, 'elite_std': params_std, 'reward_mean': rewards.mean(), 'reward_std': rewards.std()}

In [ ]:
def evaluate(env, agent, ssteps, iteration=None, episode=None, render=True):
    total_reward = 0
    obs = env.reset()
    
    for _ in itertools.islice(itertools.coint(), steps):
        action = np.asscalar(agent.act(obs))
        obs, reward, done, info = env.step(action)
        total_reward += reward
        if render: env.render()
        if done: break
            
    print('\nTeration {} episode {}: {:.2f}\n'.format(iteration, episode, total_reward))
    return total_reward

In [ ]:
def main():
    env = MarketEnv(("ES", "FUT", "GLOBEX", "USD"), obs_xform=BinaryDelta(3), episode_steps=STEPS_PER_EPISODE, client_id=2)
    obs_size = env.observation_space.shape[0]
    
    def evaluate_params(agent_params, iteration=None, episode=None):
        agent = ContinuousActionLinearPolicy(agent_params, obs_size, n_out=1)
        return evaluate(env, agent, steps=None, iteration=iteration, episode=episode)
    
    iteration = 0
    init_mean = np.zeros(obs_size + 1)
    init_std = 1.0
    for stats in cem(lambda params, episode: evaluate_params(params, iteration, episode), init_mean, params_std=init_std, n_iter=ITERATIONS, batch_size=EPISODES, elite_frac=0.2):
        timestamp = datetime.datetime.utcnow().replace(microsecond=0)
        print('\n\nITERATION {:2d} REWARD {:.2f}\n\n'.format(iteration, stats['reward_mean'], stats['reward_std']))
        print(timestamp)
        print('Best reward {:4.2f}'.format(stats['reward_best']))
        print('Best params : {}'.format(repr(stats['params_best'])))
        print('Top reward  {:4.2f}'.format(stats['reward_elite']))
        print('Top params mean : {}'.format(repr(stats['elite_mean'])))
        print('Top params std  : {}'.format(repr(stats['elite_std'])))
        